In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rwr_functions import *
from constants import *
import seaborn as sns
import matplotlib.pyplot as plt

from networkx import pagerank, from_pandas_edgelist

In [ ]:
results = []
results_ratio = []
for netname in NETWORKS:
    graph = load_graph_nx(netname)
    
    for disease in tqdm(DISEASES):
        pegasus_data = load_pegasus_results(disease)
        pagerank_seeds = init_rwr_scores_nx(graph, pegasus_data)
        gene_seeds_ncbi, ncbi_targets = load_seeds_and_targets(disease)
        
        ncbi_targets = list(set(ncbi_targets))
        for alpha in tqdm(ALPHAS):
            rwr_scores = perform_rwr_nx(alpha, graph, pagerank_seeds)
            rwr_results = process_rwr_results_nx(rwr_scores, graph, pegasus_data, pagerank_seeds) 
            
            for K in Ks:
                metrics = calculate_metrics(rwr_results, K, gene_seeds_ncbi, ncbi_targets, netname, alpha, disease)
                results.append(metrics)
                
results = pd.concat(results)
results

In [ ]:
os.makedirs("outputs", exist_ok=True)
results.to_csv("outputs/gene_scores_single_net_metrics.csv", index=False)